In [1]:
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
import math
import matplotlib.pyplot as plt

def saveTrainData (path,loop):
    temppath = path
    trainDataset = []
    for i in range(1,loop):
        path = temppath + str(i) +").png"
        img = Image.open(path)
        img = img.convert('L')
        img.load()
        trainData = np.asarray(img,dtype="float32")
        trainDataset.append(list(trainData))

    trainData = np.array(trainDataset)
    return trainData

def saveTestData(path,loop):
    temppath = path
    testDataset = []
    for i in range(1,loop):
        path = temppath + str(i) +").png"
        img = Image.open(path)
        img = img.convert('L')
        img.load()
        testData = np.asarray(img,dtype="float32")
        testDataset.append(list(testData))

    testData = np.array(testDataset)
    return testData

trainData = list(saveTrainData("C:/Users/AllenVarden/Perodua Car Recognition System/128px/Axia/Train/(",28))
trainDataTemp = list(saveTrainData("C:/Users/AllenVarden/Perodua Car Recognition System/128px/Myvi/Train/(",30))
trainData += trainDataTemp
trainDataTemp = list(saveTrainData("C:/Users/AllenVarden/Perodua Car Recognition System/128px/Viva/Train/(",31))
trainData += trainDataTemp

testData = list(saveTestData("C:/Users/AllenVarden/Perodua Car Recognition System/128px/Axia/Test/(",18))
testDataTemp = list(saveTestData("C:/Users/AllenVarden/Perodua Car Recognition System/128px/Myvi/Test/(",20))
testData += testDataTemp
testDataTemp = list(saveTestData("C:/Users/AllenVarden/Perodua Car Recognition System/128px/Viva/Test/(",20))
testData += testDataTemp

trainData =np.array(trainData)
testData = np.array(testData)

#Labels

targetData = []
targetDataNames = []
for i in range(1,4):
    if i==1:
        targetValue=0
        target="Axia"
        for j in range(1,28):
            targetData.append(targetValue)
            targetDataNames.append(target)

    elif i==2:
        targetValue=1
        target="Myvi"
        for j in range(1,30):
            targetData.append(targetValue)
            targetDataNames.append(target)

    elif i==3:
        targetValue=2
        target="Viva"
        for j in range(1,31):
            targetData.append(targetValue)
            targetDataNames.append(target)
        
 
        
trueData = []
trueDataNames = []
for i in range(1,4):
    if i==1:
        trueValue=0
        true="Axia"
        for j in range(1,18):
            trueData.append(trueValue)
            trueDataNames.append(true)
   
    elif i==2:
        trueValue=1
        true="Myvi"
        for j in range(1,20):
            trueData.append(trueValue)
            trueDataNames.append(true)
            
    elif i==3:
        trueValue=2
        true="Viva"
        for j in range(1,20):
            trueData.append(trueValue)
            trueDataNames.append(true)
            

# PCA
n_components =5
nn=3
nsamples, nx, ny = trainData.shape
trainData = trainData.reshape((nsamples,nx*ny))

pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(trainData)

eigenfaces = pca.components_

X_train_pca = pca.transform(trainData)
nsamples, nx, ny = testData.shape
testData = testData.reshape((nsamples,nx*ny))
X_test_pca = pca.transform(testData)

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

intra_class_dist = np.sum(np.power((X_train_pca[1,:] - X_test_pca[1,:]),2))
inter_class_dist = np.sum(np.power((X_train_pca[1,:] - X_test_pca[6,:]),2))
print("Intra-class distance: %d" % (intra_class_dist))
print("Inter-class distance: %d" % (inter_class_dist))

np.savetxt('train_data128.txt', trainData)
np.savetxt('test_data128.txt', testData)



# X_TR = trainData
# X_TT = testData
# Y_TR = targetData
# Y_TT = trueData

# Model 1
# Data -> PCA -> K-NN

# K-NN K=1       
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_pca,targetData)

print("\n Data -> PCA -> K-NN \n")
print("When k = 1 : training accuracy for PCA : ",round(knn.score(X_train_pca,targetData)*100,4))
print("When k = 1 : testing accuracy for PCA  : ",round(knn.score(X_test_pca,trueData)*100,4))

# Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
Y_test_pred=knn.predict(X_test_pca)
#print(Y_test_pred)
print(confusion_matrix(trueData,Y_test_pred))

# Classification report
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(3)]
print(classification_report(trueData, Y_test_pred, target_names=target_names))

# K-NN K>1       

knn=KNeighborsClassifier(n_neighbors=nn)
knn.fit(X_train_pca,targetData)

print("When k = 3 : training accuracy for PCA :",round(knn.score(X_train_pca,targetData)*100,4))
print("When k = 3 : training accuracy for PCA : ",round(knn.score(X_test_pca,trueData)*100,4))


# Confusion Matrix

Y_test_pred=knn.predict(X_test_pca)
#print(Y_test_pred)
print(confusion_matrix(trueData,Y_test_pred))

# Classification report
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(3)]
print(classification_report(trueData, Y_test_pred, target_names=target_names))



# Model 2
# Data -> LDA -> K-NN

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as l
lda=l(n_components=n_components)
lda.fit(trainData,targetData)
X_train_lda=lda.transform(trainData)
X_test_lda=lda.transform(testData)
#print(np.shape(X_train_lda))
#print(np.shape(X_test_lda))


# K-NN K=1       

knn=KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_lda,targetData)

print("\n Data -> LDA -> K-NN \n")
print("When k = 1 : training accuracy for LDA :",round(knn.score(X_train_lda,targetData)*100,4))
print("When k = 1 : training accuracy for LDA :",round(knn.score(X_test_lda,trueData)*100,4))

# Confusion Matrix

Y_test_pred=knn.predict(X_test_lda)
#print(Y_test_pred)
print(confusion_matrix(trueData,Y_test_pred))

# Classification report
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(3)]
print(classification_report(trueData, Y_test_pred, target_names=target_names))

# K-NN K>1       

knn=KNeighborsClassifier(n_neighbors=nn)
knn.fit(X_train_lda,targetData)

print("When k = 3 : training accuracy for LDA :",round(knn.score(X_train_lda,targetData)*100,4))
print("When k = 3 : testing accuracy for LDA  :",round(knn.score(X_test_lda,trueData)*100,4))

# Confusion Matrix

Y_test_pred=knn.predict(X_test_lda)
#print(Y_test_pred)
print(confusion_matrix(trueData,Y_test_pred))

# Classification report
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(3)]
print(classification_report(trueData, Y_test_pred, target_names=target_names))



# Model 3
# Data -> PCA-> LDA -> K-NN
print("\n Data -> PCA -> LDA -> K-NN \n")

lda=l(n_components=n_components)
lda.fit(X_train_pca,targetData)
X_train_pcalda=lda.transform(X_train_pca)
X_test_pcalda=lda.transform(X_test_pca)
print(np.shape(X_train_pcalda))
print(np.shape(X_test_pcalda))


# K-NN K=1       

knn=KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_pcalda,targetData)


print("When k = 1 : training accuracy for PCA and LDA :",round(knn.score(X_train_pcalda,targetData)*100,4))
print("When k = 1 : testing accuracy for PCA and LDA  : ",round(knn.score(X_test_pcalda,trueData)*100,4))

# Confusion Matrix

Y_test_pred=knn.predict(X_test_pcalda)
#print(Y_test_pred)
print(confusion_matrix(trueData,Y_test_pred))

# Classification report
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(3)]
print(classification_report(trueData, Y_test_pred, target_names=target_names))

# K-NN K>1       

knn=KNeighborsClassifier(n_neighbors=nn)
knn.fit(X_train_pcalda,targetData)

print("When k = 3 : training accuracy for PCA and LDA : ",round(knn.score(X_train_pcalda,targetData)*100,4))
print("When k = 3 : training accuracy for PCA and LDA :",round(knn.score(X_test_pcalda,trueData)*100,4))

# Confusion Matrix

Y_test_pred=knn.predict(X_test_pcalda)
#print(Y_test_pred)
print(confusion_matrix(trueData,Y_test_pred))

# Classification report
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(3)]
print(classification_report(trueData, Y_test_pred, target_names=target_names))


# Model 4
# Data -> Neural Network


# Preprocessing :Data normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(trainData)
# Normalization
X_train=scaler.transform(trainData)
X_test=scaler.transform(testData)

# Perceptron
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(100,100,100,100),activation='relu',random_state=0,max_iter=100,learning_rate_init=0.001)
# Perceptron training(fit function)
mlp.fit(X_train,targetData)
# Performance Analysis
print("\n Data -> NN \n")
print("With 4 hidden layers, the training accuracy for NN : ",round(mlp.score(X_train,targetData)*100,2))
print("With 4 hidden layers, the testing accuracy for NN  : ",round(mlp.score(X_test,trueData)*100,2))

# Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
Y_test_pred=mlp.predict(X_test)
#print(Y_test_pred)
print(confusion_matrix(trueData,Y_test_pred))

# Classification report
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(3)]
print(classification_report(trueData, Y_test_pred, target_names=target_names))


<Figure size 640x480 with 1 Axes>

Intra-class distance: 9
Inter-class distance: 18

 Data -> PCA -> K-NN 

When k = 1 : training accuracy for PCA :  100.0
When k = 1 : testing accuracy for PCA  :  49.0909
[[ 9  2  6]
 [ 3  8  8]
 [ 2  7 10]]
             precision    recall  f1-score   support

    Class 0       0.64      0.53      0.58        17
    Class 1       0.47      0.42      0.44        19
    Class 2       0.42      0.53      0.47        19

avg / total       0.51      0.49      0.49        55

When k = 3 : training accuracy for PCA : 68.6047
When k = 3 : training accuracy for PCA :  32.7273
[[8 5 4]
 [5 5 9]
 [7 7 5]]
             precision    recall  f1-score   support

    Class 0       0.40      0.47      0.43        17
    Class 1       0.29      0.26      0.28        19
    Class 2       0.28      0.26      0.27        19

avg / total       0.32      0.33      0.32        55


 Data -> LDA -> K-NN 

When k = 1 : training accuracy for LDA : 100.0
When k = 1 : training accuracy for LDA : 38.1818
[[6 5 6]


D:\Anaconda3\lib\site-packages\numpy\core\_methods.py:138: RuntimeWarning: invalid value encountered in sqrt
  ret = um.sqrt(ret, out=ret)
D:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
D:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:653: RuntimeWarning: invalid value encountered in sqrt
  self.scale_ = _handle_zeros_in_scale(np.sqrt(self.var_))



 Data -> NN 

With 4 hidden layers, the training accuracy for NN :  100.0
With 4 hidden layers, the testing accuracy for NN  :  47.27
[[ 7  5  5]
 [ 7  9  3]
 [ 3  6 10]]
             precision    recall  f1-score   support

    Class 0       0.41      0.41      0.41        17
    Class 1       0.45      0.47      0.46        19
    Class 2       0.56      0.53      0.54        19

avg / total       0.47      0.47      0.47        55

